In [11]:
import cv2
import os
import face_recognition as fr
import numpy as np
import csv
from datetime import datetime

In [12]:
path='images'

In [13]:
image_list=os.listdir(path)
image_list

['Aswin_M_S.jpg',
 'Declan Rice.jpg',
 'Gal Gadot.jpg',
 'Messi.jpg',
 'Odegaard.jpg',
 'Rowan Atkinson.jpg']

In [14]:
img_input=[]
classnames=[]
for i in image_list:
    img_path=os.path.join(path,i)
    img=cv2.imread(img_path)
    img_input.append(img)
    classnames.append(i.split('.')[0])

In [15]:
img_input

[array([[[124, 124, 124],
         [126, 126, 126],
         [125, 125, 125],
         ...,
         [124, 124, 124],
         [126, 126, 126],
         [124, 124, 124]],
 
        [[126, 126, 126],
         [254, 254, 254],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [254, 254, 254],
         [125, 125, 125]],
 
        [[125, 125, 125],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [125, 125, 125]],
 
        ...,
 
        [[125, 124, 126],
         [ 74,  67,  74],
         [ 72,  65,  70],
         ...,
         [ 50,  45,  47],
         [ 56,  49,  52],
         [125, 125, 125]],
 
        [[125, 125, 125],
         [ 66,  59,  66],
         [ 71,  66,  68],
         ...,
         [ 55,  50,  52],
         [ 58,  51,  54],
         [127, 125, 125]],
 
        [[123, 125, 125],
         [127, 123, 128],
         [126, 124, 124],
         ...,
         [125, 125, 125],
  

In [16]:
classnames

['Aswin_M_S',
 'Declan Rice',
 'Gal Gadot',
 'Messi',
 'Odegaard',
 'Rowan Atkinson']

In [17]:
def encoding(images):
    encoded_list=[]
    for img in images:
        img1=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        face_loc=fr.face_locations(img)
        face_enc=fr.face_encodings(img1,face_loc)[0]
        encoded_list.append(face_enc)
    return encoded_list

In [18]:
encoded_know_face=encoding(img_input)
print(encoded_know_face)

[array([-0.11385319,  0.08346479,  0.03443676, -0.03870995,  0.05433658,
        0.01565935, -0.0053499 , -0.01531149,  0.20329224, -0.03885418,
        0.16787226,  0.0562616 , -0.20547312, -0.06901471, -0.01148995,
        0.05425726, -0.11899826, -0.17287743, -0.09220014, -0.07468627,
        0.02679684,  0.08729014, -0.05547515,  0.021937  , -0.21768051,
       -0.28627175, -0.01418749, -0.22967283,  0.00998082, -0.12048896,
       -0.05100938, -0.0468284 , -0.21796957, -0.01422155, -0.01159918,
        0.07886404,  0.04555615, -0.01368058,  0.17531824,  0.07099718,
       -0.10266206, -0.0371189 ,  0.03332542,  0.25981322,  0.12715733,
        0.02671305,  0.03556072, -0.03857294,  0.12110363, -0.23089339,
        0.144934  ,  0.07211329,  0.14282949,  0.06599309,  0.12000351,
       -0.22005674, -0.02330314, -0.01640742, -0.1943019 ,  0.13926943,
        0.04972708,  0.0190193 , -0.12066273, -0.01127402,  0.20340446,
        0.04752155, -0.09805629, -0.0963818 ,  0.16734529, -0.1

In [19]:
atte_file = "attendance.csv"

def mark_attendance(name):
    now = datetime.now()
    date_today = now.strftime("%d-%m-%Y")
    time_now = now.strftime("%H:%M:%S")

    if not os.path.exists(atte_file):
        with open(atte_file, "w", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(["Name", "Date", "Time"])

    with open(atte_file, "r", newline="") as f:
        reader = csv.reader(f)
        records = list(reader)

    for row in records[1:]:  
        if row[0] == name and row[1] == date_today:
            print(f"Attendance already marked for {name} today")
            return False

    with open(atte_file, "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([name, date_today, time_now])

    print(f"Attendance marked for {name} at {time_now}")
    return True

In [20]:
video=cv2.VideoCapture(0)
while True:
    suc,frame=video.read()
    frame1=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    face_loca=fr.face_locations(frame1)
    face_enco=fr.face_encodings(frame1,face_loca)
    for encoder,loc in zip(face_enco,face_loca):
        match=fr.compare_faces(encoded_know_face,encoder)
        distance=fr.face_distance(encoded_know_face,encoder)
        index=np.argmin(distance)
        if match[index]:
            name=classnames[index]
            mark_attendance(name)
        else:
            name='not enrolled'
            print(f'You are {name}, please contact admin.')
        y1,x2,y2,x1=loc
        cv2.rectangle(frame,(x1,y1),(x2,y2),(0,0,255),2)
        cv2.putText(frame,name,(x1,y1),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
    cv2.imshow("Webcam",frame)
    if cv2.waitKey(1) & 0XFF==ord('q'):
        break
video.release()
cv2.destroyAllWindows()


Attendance marked for Aswin_M_S at 20:27:41
Attendance already marked for Aswin_M_S today
Attendance already marked for Aswin_M_S today
Attendance already marked for Aswin_M_S today
Attendance already marked for Aswin_M_S today
Attendance already marked for Aswin_M_S today
Attendance already marked for Aswin_M_S today
Attendance marked for Odegaard at 20:27:49
Attendance already marked for Odegaard today
Attendance already marked for Odegaard today
Attendance already marked for Odegaard today
Attendance already marked for Aswin_M_S today
